In [1]:
import tweepy
import json
import pandas as pd
import csv
from os import path
#from textblob import TextBlob
#from textblob import Word

In [2]:
consumer_key = '______________'
consumer_secret = '_______________'
access_token = '_______________'
access_token_secret = '___________________'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
extracted_tweets = []
search_filter = "-filter:retweets"

In [3]:
def extract_tweets(search_filter):
    try:
        get_tweets = []
        for tweet in tweepy.Cursor (api.search, q = search_filter,lang = "en").items(10): 
            get_tweets.append(tweet)
        return get_tweets
    except:
        print("Time out error caught.")

In [4]:
def tweets_df(raw_tweets):
    #Select specified data from JSON format of tweet to Dataframe
    id_list = [tweet.id for tweet in raw_tweets]
    organised_tweets = pd.DataFrame(id_list, columns = ["id"])
    organised_tweets["text"] = [tweet.text for tweet in raw_tweets]
    organised_tweets["created_at"] = [tweet.created_at for tweet in raw_tweets]
    organised_tweets["retweet_count"] = [tweet.retweet_count for tweet in raw_tweets]
    organised_tweets["user_location"] = [tweet.author.location for tweet in raw_tweets]
    organised_tweets["Hashtags"] = [tweet.entities.get('hashtags') for tweet in raw_tweets]
    return organised_tweets

In [5]:
def organize_data(raw_tweets):
    organised_tweets_df = tweets_df(raw_tweets)
    text = organised_tweets_df["text"]
    #Remove https link from tweets
    for i in range(0,len(text)):
        txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
        organised_tweets_df.at[i, 'text2'] = txt
    # Remove tweets with duplicate text
    organised_tweets_df.drop_duplicates('text2', inplace=True)
    organised_tweets_df.reset_index(drop = True, inplace=True)
    organised_tweets_df.drop('text', axis = 1, inplace = True)
    organised_tweets_df.rename(columns={'text2': 'text'}, inplace=True)
    return organised_tweets_df

In [6]:
def create_csv():
    extracted_tweets = extract_tweets(search_filter)
    sample_data = organize_data(extracted_tweets)
    sample_data.to_csv(r'Twitter_Data.csv', index = False)

In [7]:
def append_tocsv():
    extracted_tweets = extract_tweets(search_filter)
    sample_data = organize_data(extracted_tweets)
    current_data = pd.read_csv('Twitter_Data.csv')
    sample_data = sample_data[sample_data['created_at'] >= current_data.created_at.max()]
    sample_data.to_csv(r'Twitter_Data.csv', index = False, mode = 'a', header = False)

In [8]:
if path.exists('Twitter_Data.csv'):
    append_tocsv()
else:
    create_csv()